In [ ]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

# 검색 키워드와 페이지 범위 설정
search_keyword = "물티슈"
start_page = 1
end_page = 5

# 전체 결과 저장 리스트
all_data = []

# 브라우저 실행
driver = uc.Chrome(headless=False)

for page in range(start_page, end_page + 1):
    url = (
        f"https://www.coupang.com/np/search?component=&q={search_keyword}"
        f"&channel=user&sorter=scoreDesc&listSize=36&page={page}"
    )
    print(f"📄 {page}페이지 크롤링 중...")
    driver.get(url)

    # 상품 블록 로드될 때까지 대기
    try:
        WebDriverWait(driver, 15).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "li.ProductUnit_productUnit__Qd6sv"))
        )
    except:
        print(f"⚠ {page}페이지 로딩 실패, 건너뜀")
        continue

    # 끝까지 스크롤 (스크롤 후 대기)
    scroll_pause_time = 2
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause_time)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # HTML 파싱
    soup = BeautifulSoup(driver.page_source, "html.parser")
    products = soup.select("li.ProductUnit_productUnit__Qd6sv")
    print(f"   ➡ {len(products)}개 상품 로드됨")

    # 상품 정보 추출
    for p in products:
        name_tag = p.select_one("div.ProductUnit_productName__gre7e")
        price_tag = p.select_one("strong.Price_priceValue__A4KOr")
        review_tag = p.select_one("span.ProductRating_ratingCount__R0Vhz")
        link_tag = p.find("a", href=True)  # 클래스 상관없이 첫 <a> 태그
        
        link = "https://www.coupang.com" + link_tag["href"] if link_tag else None
        
        # 이름과 링크만 필수, 가격/리뷰 없으면 "0"
        if name_tag and link:
            all_data.append({
                "제품명": name_tag.get_text(strip=True),
                "가격": price_tag.get_text(strip=True) if price_tag else "0",
                "리뷰수": review_tag.get_text(strip=True).strip("()") if review_tag else "0",
                "링크": link
            })

# 브라우저 종료
driver.quit()

# 중복 제거 후 CSV 저장
df = pd.DataFrame(all_data).drop_duplicates(subset=["링크"])
df.to_csv("coupang_wet_tissue_rank36_pages1to5.csv", index=False, encoding="utf-8-sig")
print(f"✅ 크롤링 완료! 총 {len(df)}개 상품 저장됨 (중복 제거됨)")